In [7]:

import psycopg2
import pandas as pd
 
# # connection establishment
# conn = psycopg2.connect(
#    database="postgres",
#     user='postgres',
#     password='password',
#     host='localhost',
#     port= '5432'
# )
 
# conn.autocommit = True
 
# # Creating a cursor object
# cursor = conn.cursor()
 
# # query to create a database 
# sql = ''' CREATE database ppdb '''; 
# # executing above query
# cursor.execute(sql)
# print("Database has been created successfully !!")

# createTable(cursor)

# parse_file('ppdb-2.0-xxxl-all',cursor)
 
# # Closing the connection
# conn.close()


In [8]:
def get_ppdb2_score(array):
    features = array[3].split(' ')
    for feature in features:
        if feature.startswith('PPDB2'):
            return feature.split('=')[1]


def get_ppdb1_score(array):
    features = array[3].split(' ')
    for feature in features:
        if feature.startswith('PPDB1'):
            return feature.split('=')[1]
            
def parse_file(file_path, cursor,conn):

    # Open the file for reading
    with open(file_path, 'r') as file:
        # Read the file line by line
        errorLines = []
        count = 0
        for line in file:
            # Strip newline characters and split the line into columns
            columns = line.strip().split('|||')
            columns[0] = columns[0].replace(']','').replace('[','')
            # print (columns)
            p1score = get_ppdb1_score(columns)
            # print (p1score)
            p2score = get_ppdb2_score(columns)
            # print (p2score)
            # Add the parsed columns to the parsed_data list
            # cursor.execute("INSERT INTO ppdb VALUES ({count}, '{columns[0]}', '{columns[1]}', '{columns[2]}', {p1score}, {p2sore}, '{columns[4]}', '{columns[5]}')".format(count=count, columns=columns, p1score=p1score, p2sore=p2score))
            try:
                cursor.execute("INSERT INTO ppdb VALUES (%s, %s, %s, %s, %s, %s, %s, %s)", (count, columns[0], columns[1], columns[2], p1score, p2score, columns[4], columns[5]))
                conn.commit()
            except Exception as e:
                print (e)
                errorLines.append(line)

            count += 1
            return errorLines

In [9]:
def createTable(cursor):
    cursor.execute("""CREATE TABLE if not exists ppdb (id SERIAL PRIMARY KEY, lhs TEXT, phrase TEXT, paraphrase TEXT, ppdb1_score FLOAT4, ppdb2_score FLOAT4, alignment TEXT, entailment TEXT);
                   TRUNCATE TABLE ppdb;""")

In [10]:
conn = psycopg2.connect(
   database="ppdb",
    user='postgres',
    password='password',
    host='localhost',
    port= '5432'
)
 
# Creating a cursor object
cursor = conn.cursor()

createTable(cursor)
print("Table has been created successfully !!")

issueLines = parse_file('ppdb-2.0-xxxl-all', cursor, conn)
df = pd.DataFrame(issueLines)
df.to_csv('issueLines.csv', index=False)
print("Data has been parsed successfully !!")
cursor.close()
 
# Closing the connection
conn.close()